# Step 2 - Consume your data through the python connector and dump it in mongodb atlas.

In [1]:
import argparse

from confluent_kafka import Consumer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.serialization import SerializationContext, MessageField
from confluent_kafka.schema_registry.json_schema import JSONDeserializer

In [3]:
# Variables
API_KEY = '***Removed while uploading code to github, add again to run*****'
ENDPOINT_SCHEMA_URL  = 'https://psrc-8qyy0.eastus2.azure.confluent.cloud'
API_SECRET_KEY = '***Removed while uploading code to github, add again to run*****'
BOOTSTRAP_SERVER = 'pkc-n00kk.us-east-1.aws.confluent.cloud:9092'
SECURITY_PROTOCOL = 'SASL_SSL'
SSL_MACHENISM = 'PLAIN'
SCHEMA_REGISTRY_API_KEY = 'BS3PHBXGYPRUDGFP'
SCHEMA_REGISTRY_API_SECRET = '***Removed while uploading code to github, add again to run*****'


In [4]:
def sasl_conf():

    sasl_conf = {'sasl.mechanism': SSL_MACHENISM,
                 # Set to SASL_SSL to enable TLS support.
                #  'security.protocol': 'SASL_PLAINTEXT'}
                'bootstrap.servers':BOOTSTRAP_SERVER,
                'security.protocol': SECURITY_PROTOCOL,
                'sasl.username': API_KEY,
                'sasl.password': API_SECRET_KEY
                }
    return sasl_conf

def schema_config():
    return {'url':ENDPOINT_SCHEMA_URL,
    'basic.auth.user.info':f"{SCHEMA_REGISTRY_API_KEY}:{SCHEMA_REGISTRY_API_SECRET}"
    }

class Car:   
    def __init__(self,record:dict):
        for k,v in record.items():
            setattr(self,k,v)
        
        self.record=record
   
    @staticmethod
    def dict_to_car(data:dict,ctx):
        return Car(record=data)

    def __str__(self):
        return f"{self.record}"


def kafka_consumer(topic, schema_id):
    schema_registry_conf = schema_config()
    schema_registry_client = SchemaRegistryClient(schema_registry_conf)

    print('reading schema')
    schema_str = schema_registry_client.get_schema(schema_id).schema_str
    print(schema_str)
    json_deserializer = JSONDeserializer(schema_str,
                                         from_dict=Car.dict_to_car)

    print("deserializer start")
    consumer_conf = sasl_conf()
    consumer_conf.update({
                     'group.id': 'group1',
                     'auto.offset.reset': "latest"})

    print('consumer')
    consumer = Consumer(consumer_conf)
    consumer.subscribe([topic])


    while True:
        try:
            # SIGINT can't be handled when polling, limit timeout to 1 second.
            msg = consumer.poll(1.0)
            if msg is None:
                continue

            car = json_deserializer(msg.value(), SerializationContext(msg.topic(), MessageField.VALUE))

            if car is not None:
                print("User record {}: car: {}\n"
                      .format(msg.key(), car))
        except KeyboardInterrupt:
            break

    consumer.close()

## Case CSV file

In [7]:
topic = 'topic_case'
schema_id = 100005

In [ ]:
kafka_consumer(topic, schema_id)

## Patientinfo CSV file

In [5]:
topic = 'topic_patientinfo'
schema_id = 100007

In [ ]:
kafka_consumer(topic, schema_id)

## Policy CSV file

In [10]:
topic = 'topic_policy'
schema_id = 100008

In [ ]:
kafka_consumer(topic, schema_id)

## Region CSV file

In [12]:
topic = 'topic_region'
schema_id = 100009

In [ ]:
kafka_consumer(topic, schema_id)

## SearchTrend CSV file

In [14]:
topic = 'topic_searchtrend'
schema_id = 100010

In [ ]:
kafka_consumer(topic, schema_id)

## SeoulFloating CSV file

In [16]:
topic = 'topic_seoulfloating'
schema_id = 100011

In [ ]:
kafka_consumer(topic, schema_id)

## Time CSV file

In [19]:
topic = 'topic_time'
schema_id = 100012

In [ ]:
kafka_consumer(topic, schema_id)

## TimeAge CSV file

In [21]:
topic = 'topic_timeage'
schema_id = 100013

In [ ]:
kafka_consumer(topic, schema_id)

## TimeGender CSV file

In [23]:
topic = 'topic_timegender'
schema_id = 100014

In [ ]:
kafka_consumer(topic, schema_id)

## TimeProvince CSV file

In [25]:
topic = 'topic_timeprovince'
schema_id = 100015

In [ ]:
kafka_consumer(topic, schema_id)

## Weather CSV file

In [27]:
topic = 'topic_weather'
schema_id = 100016

In [ ]:
kafka_consumer(topic, schema_id)